In [1]:
# 기본
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import matplotlib
from matplotlib import font_manager, rc
import platform
from tqdm import tqdm
import sklearn
from sklearn import linear_model
import scipy.stats as stats
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor
from patsy import dmatrices
from sklearn.model_selection import train_test_split

# time
import datetime

# crawling
import requests
import lxml.html
import sqlite3
from pandas.io import sql
from bs4 import BeautifulSoup

# 한글 폰트 설정
if platform.system() == 'Windows':
# 윈도우인 경우
    font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
    rc('font', family=font_name)

In [2]:
# 관측소 코드 담기

where = 94866   # Melbourne

In [33]:
where = 72402   # WAL Wallops Island Observations

In [34]:
final = pd.DataFrame()

# 30일, 31일 월 분리
# 2월은 따로 생각해야 함
months_30 = [4, 6, 9, 11]
months_31 = [1, 3, 5, 7, 8, 10, 12]

for year in range(2014, 2018):   # 여기 바꾸기
    for month in range(1, 13): 
        if month in months_30: 
            response = requests.get('https://weather.uwyo.edu/cgi-bin/sounding?region=seasia&TYPE=TEXT%3ALIST&YEAR={}&MONTH={}&FROM=0100&TO=3012&STNM={}'.format(year, month, where), verify=False)
        elif month in months_31: 
            response = requests.get('https://weather.uwyo.edu/cgi-bin/sounding?region=seasia&TYPE=TEXT%3ALIST&YEAR={}&MONTH={}&FROM=0100&TO=3112&STNM={}'.format(year, month, where), verify=False)
        else:   # 2월인 경우
            if year % 4 == 0:   # 윤년인 경우 따로 분리 - 29일까지 있음
                response = requests.get('https://weather.uwyo.edu/cgi-bin/sounding?region=seasia&TYPE=TEXT%3ALIST&YEAR={}&MONTH={}&FROM=0100&TO=2912&STNM={}'.format(year, month, where), verify=False)
            else:   # 28일까지 있는 경우
                response = requests.get('https://weather.uwyo.edu/cgi-bin/sounding?region=seasia&TYPE=TEXT%3ALIST&YEAR={}&MONTH={}&FROM=0100&TO=2812&STNM={}'.format(year, month, where), verify=False)
        
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')

        title = soup.select('body > h2')  # title은 하나씩 있음
        data = soup.select('body > pre')  # pre는 하나 건너 하나씩 concat 해야함

        idx_data = 0
        for idx in range(len(title)): 
            data_df = pd.DataFrame(data[idx_data])
            data_df[0] = data_df[0].str.replace("-----------------------------------------------------------------------------", "")
            data_df_2 = data_df[0].str.split('\n', expand=True)

            data_df_2.drop([0, 1, 2, 3, 4], axis = 1, inplace = True)

            data_df_2 = data_df_2.transpose()
            title_tmp = str(title[idx])
            data_df_2['date'] = title_tmp[4:60]   # 멜버른 기준 4:64, wall wallops island 기준 4:60
            
            final = pd.concat([final, data_df_2], axis = 0)

            idx_data += 2

c:\Users\user\anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'weather.uwyo.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
c:\Users\user\anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'weather.uwyo.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
c:\Users\user\anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'weather.uwyo.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
c:\Users\user\anaconda3\lib\site-packages\urllib3\connecti

In [ ]:
# WAL Wallops Island 14-17 돌리는 중

In [29]:
title_tmp[4:60]

'72402 WAL Wallops Island Observations at 12Z 31 Dec 2013'

In [30]:
final

,0,date
5,1019.0 12 6.0 -3.0 52 3.02 ...,72402 WAL Wallops Island Observations at 00Z 0...
6,1002.0 145 4.8 -4.2 52 2.81 ...,72402 WAL Wallops Island Observations at 00Z 0...
7,1000.0 161 4.8 -4.2 52 2.81 ...,72402 WAL Wallops Island Observations at 00Z 0...
8,982.5 305 5.0 -7.7 39 2.19 ...,72402 WAL Wallops Island Observations at 00Z 0...
9,981.0 318 5.0 -8.0 38 2.14 ...,72402 WAL Wallops Island Observations at 00Z 0...
...,...,...
111,7.0 33230 -36.3 -66.3 3 0.73 ...,72402 WAL Wallops Island Observations at 12Z 3...
112,6.7 33528 -36.7 -66.7 3 0.72 ...,72402 WAL Wallops Island Observations at 12Z 3...
113,6.4 33833 -37.1 -67.1 3 0.71 ...,72402 WAL Wallops Island Observations at 12Z 3...
114,6.3 33959 -37.3 -67.3 3 0.71 ...,72402 WAL Wallops Island Observations at 12Z 3...


In [31]:
final2 = final.reset_index(drop = True)
final2

,0,date
0,1019.0 12 6.0 -3.0 52 3.02 ...,72402 WAL Wallops Island Observations at 00Z 0...
1,1002.0 145 4.8 -4.2 52 2.81 ...,72402 WAL Wallops Island Observations at 00Z 0...
2,1000.0 161 4.8 -4.2 52 2.81 ...,72402 WAL Wallops Island Observations at 00Z 0...
3,982.5 305 5.0 -7.7 39 2.19 ...,72402 WAL Wallops Island Observations at 00Z 0...
4,981.0 318 5.0 -8.0 38 2.14 ...,72402 WAL Wallops Island Observations at 00Z 0...
...,...,...
77271,7.0 33230 -36.3 -66.3 3 0.73 ...,72402 WAL Wallops Island Observations at 12Z 3...
77272,6.7 33528 -36.7 -66.7 3 0.72 ...,72402 WAL Wallops Island Observations at 12Z 3...
77273,6.4 33833 -37.1 -67.1 3 0.71 ...,72402 WAL Wallops Island Observations at 12Z 3...
77274,6.3 33959 -37.3 -67.3 3 0.71 ...,72402 WAL Wallops Island Observations at 12Z 3...


In [32]:
final2.to_csv('D:\\SNUlab\\0. data\\thermal inversion\\WAL_Wallops_Island_crawling_13.csv', encoding = 'euc-kr', index = False)